In [2]:
from google.colab import drive
drive.mount('/content/drive')

from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2TokenizerFast
import numpy as np
from torch.cuda.amp import autocast, GradScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

class CasualSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=False)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=False)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.block_size, config.block_size))
            .view(1, 1, config.block_size, config.block_size),
        )

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        att = (q @ k.transpose(-2, -1)) * (1.0 / (C // self.n_head) ** 0.5)
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
        att = F.softmax(att, dim=-1)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.attention = CasualSelfAttention(config)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attention(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 256
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.2

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(
            dict(
                token_embedding=nn.Embedding(config.vocab_size, config.n_embd),
                position_embedding=nn.Embedding(config.block_size, config.n_embd),
                h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                ln_f=nn.LayerNorm(config.n_embd),
            )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        token_embeddings = self.transformer.token_embedding(idx)
        position_ids = torch.arange(T, device=idx.device).unsqueeze(0)
        position_embeddings = self.transformer.position_embedding(position_ids)
        x = token_embeddings + position_embeddings
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

class GPTDataset(torch.utils.data.Dataset):
    def __init__(self, path, block_size):
        self.data = np.memmap(path, dtype=np.uint16, mode="r")
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx : idx + self.block_size], dtype=torch.long)
        y = torch.tensor(self.data[idx + 1 : idx + 1 + self.block_size], dtype=torch.long)
        return x, y


config = GPTConfig()
model = GPT(config).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/minigpt2step_19000.pth", map_location=device))

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
dataset = GPTDataset("/content/drive/MyDrive/QA_tokens.bin", block_size=config.block_size)

batch_size = 96
loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=4
)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

num_epochs = 3
global_step = 0
save_path = "/content/drive/MyDrive/minigpt2_final_finetuned.pth"

model.train()
for epoch in range(num_epochs):
    print(f"\nStarting epoch {epoch + 1}/{num_epochs}")
    for step, (x, y) in enumerate(loader):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad()

        with autocast():
            logits, loss = model(x, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        global_step += 1
        if global_step % 50 == 0:
            print(f"Epoch {epoch+1} Step {global_step}, Loss: {loss.item():.4f}")
        if global_step % 500 == 0:
            torch.save(model.state_dict(), save_path)
            print(f" Model saved to {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Starting epoch 1/3


/tmp/ipython-input-2-2463023450.py:136: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-2-2463023450.py:149: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1 Step 50, Loss: 0.1505
Epoch 1 Step 100, Loss: 0.1112
Epoch 1 Step 150, Loss: 0.1009
Epoch 1 Step 200, Loss: 0.1169
Epoch 1 Step 250, Loss: 0.1140
Epoch 1 Step 300, Loss: 0.0895
Epoch 1 Step 350, Loss: 0.0898
Epoch 1 Step 400, Loss: 0.0915
Epoch 1 Step 450, Loss: 0.0796
Epoch 1 Step 500, Loss: 0.0781
 Model saved to /content/drive/MyDrive/minigpt2_final_finetuned.pth
Epoch 1 Step 550, Loss: 0.0741
Epoch 1 Step 600, Loss: 0.0663
Epoch 1 Step 650, Loss: 0.0659
Epoch 1 Step 700, Loss: 0.0621
Epoch 1 Step 750, Loss: 0.0589
Epoch 1 Step 800, Loss: 0.0575
Epoch 1 Step 850, Loss: 0.0533
Epoch 1 Step 900, Loss: 0.0511
Epoch 1 Step 950, Loss: 0.0503
Epoch 1 Step 1000, Loss: 0.0493
 Model saved to /content/drive/MyDrive/minigpt2_final_finetuned.pth
Epoch 1 Step 1050, Loss: 0.0476
Epoch 1 Step 1100, Loss: 0.0446
Epoch 1 Step 1150, Loss: 0.0429
Epoch 1 Step 1200, Loss: 0.0424
Epoch 1 Step 1250, Loss: 0.0394
Epoch 1 Step 1300, Loss: 0.0428
Epoch 1 Step 1350, Loss: 0.0398
Epoch 1 Step 1400, L

In [3]:
torch.save(model.state_dict(), save_path)